## Hyperdrive Exploration Simulation
This notebook is to explore the base functionality of Hyperdrive

For details on the simulation framework, please see our <a href="https://elfpy.element.fi/"> simulation documentation</a>

### Install repo requirements & import packages

In [191]:
# test: skip-cell
try: # install dependencies only if running on google colab
    import google.colab # check if running in colab
    !pip install -r https://raw.githubusercontent.com/element-fi/elf-simulations/main/requirements-3.8.txt 
    !pip install git+https://github.com/element-fi/elf-simulations.git
except:
    print("running locally & trusting that you have the dependencies installed")

running locally & trusting that you have the dependencies installed


In [192]:
from __future__ import annotations

import numpy as np
from numpy.random._generator import Generator
import matplotlib.ticker as ticker

from elfpy import WEI
from elfpy.agents.agent import Agent
from elfpy.markets.hyperdrive import Market, MarketAction, MarketActionType
from elfpy.utils import sim_utils
import elfpy.utils.outputs as output_utils
import elfpy.utils.post_processing as post_processing
from elfpy.simulators import Config
import pandas as pd


### Setup experiment parameters

In [202]:
config = Config()

# General config parameters
config.title = "Hyperdrive Core Features"
config.pricing_model_name = "Hyperdrive" # can be yieldspace or hyperdrive

config.num_trading_days = 100 # Number of simulated trading days
config.num_blocks_per_day = 5 # Blocks in a given day (7200 means ~12 sec per block)
config.num_position_days = 90 # How long a token reaches maturity

config.trade_fee_percent = 0.05 # fee percent collected on trades
config.redemption_fee_percent = 0.05 # fee collected on the spread of the flat portion

config.target_fixed_apr = .03 # target fixed APR of the initial market after the LP
config.target_liquidity = 2_000_000 # target total liquidity of the initial market, before any trades

config.log_level = output_utils.text_to_log_level("INFO") # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
config.log_filename = "./hyperdrive_exploration.log" # Output filename for logging

config.shuffle_users = True

# Notebook specific parameters
num_longs = 1
num_shorts = 1

#vault_apr[config.num_trading_days//2:] = 0.05
# Define the vault apr
vault_apr = np.array([0.01] * config.num_trading_days)
config.variable_apr = vault_apr.tolist()
config.freeze()

fig_size = (5, 5)

### Setup agents

In [194]:
class ShortAgent(Agent):
    """Opens Basic Short"""
    def action(self, market: Market):
        """implement user strategy"""

        short_balances = [short.balance for short in self.wallet.shorts.values()]
        has_opened_short = bool(any(short_balance > 0 for short_balance in short_balances))

        print("short")
        if not has_opened_short:
            print("Short Amount: %s" %(1000))
            return [
                self.create_agent_action(
                    action_type=MarketActionType.OPEN_SHORT,
                    trade_amount=100_000,
                    mint_time=market.time
                )
            ]
        else:
            return []

In [195]:
class LongAgent(Agent):
    """Opens Basic Long"""
    def action(self, market: Market):
        """implement user strategy"""
        long_balances = [long.balance for long in self.wallet.longs.values()]
        has_opened_long = bool(any(long_balance > 0 for long_balance in long_balances))

        if not has_opened_long:
            print("Long Amount: %s" %(1000))
            return [
                self.create_agent_action(
                    action_type=MarketActionType.OPEN_LONG,
                    trade_amount=1000,
                    mint_time=market.time
                )
            ]
        else:
            return []

In [196]:
class LPAgent(Agent):
    """Adds a large LP"""
    def action(self, market: Market):
        """implement user strategy"""
        if self.wallet.lp_tokens > 0: # has already opened the lp
            action_list = []
        else:
            action_list = [
                self.create_agent_action(
                    action_type=MarketActionType.ADD_LIQUIDITY, trade_amount=self.budget
                ),
            ]
        return action_list

### Instantiate Example Agents

In [197]:

def get_example_agents(rng: Generator, num_shorts: int, num_longs: int, existing_agents: int = 0) -> list[Agent]:
    """Instantiate a set of custom agents"""
    agents = []

    for address in range(existing_agents, existing_agents + num_shorts):
        agent = ShortAgent(
            wallet_address=address,
            budget=2_000_000,
        )
        agent.log_status_report()
        agents += [agent]

    for address in range(existing_agents, existing_agents + num_longs):
        agent = LongAgent(
            wallet_address=address,
            budget=1_999_000,
        )
        agent.log_status_report()
        agents += [agent]

    existing_agents += len(agents)
    return agents

### Setup simulation objects

In [198]:
# define root logging parameters
output_utils.setup_logging(log_filename=config.log_filename, log_level=config.log_level)

# get an instantiated simulator object
simulator = sim_utils.get_simulator(config)

### Run the simulation

In [199]:
bond_reserves = []
simulator = sim_utils.get_simulator(config)

# add the agents
trading_agents = get_example_agents(
    rng=simulator.rng,
    num_shorts=num_shorts,
    num_longs=num_longs,
    existing_agents=len(simulator.agents)
)
simulator.add_agents(trading_agents)
print(f"Simulator has {len(simulator.agents)} agents")

# run the simulation
simulator.run_simulation()

Simulator has 2 agents
Long Amount: 1000


In [200]:
# print(simulator.get_simulation_state_string())
# print(simulator.market.__dict__)

# convert simulation state to a pandas dataframe
trades = post_processing.compute_derived_variables(simulator)
# print(trades.iloc[1]["share_reserves"])

print("before values: \n")

print("bond reserves: %s" %(trades.iloc[0]["bond_reserves"]))
print("share reserves: %s" %(trades.iloc[0]["share_reserves"]))
print("base buffer: %s" %(trades.iloc[0]["base_buffer"]))
print("bond buffer: %s" %(trades.iloc[0]["bond_buffer"]))
print("share price: %s" %(trades.iloc[0]["share_price"]))
print("fixed apr: %s \n" %(trades.iloc[0]["fixed_apr"]))

print("after values: \n")

print("bond reserves: %s" %(trades.iloc[1]["bond_reserves"]))
print("share reserves: %s" %(trades.iloc[1]["share_reserves"]))
print("base buffer: %s" %(trades.iloc[1]["base_buffer"]))
print("bond buffer: %s" %(trades.iloc[1]["bond_buffer"]))
print("share price: %s" %(trades.iloc[1]["share_price"]))
print("fixed apr: %s \n" %(trades.iloc[1]["fixed_apr"]))

print("after close: \n")

print("bond reserves: %s" %(trades.iloc[2]["bond_reserves"]))
print("share reserves: %s" %(trades.iloc[2]["share_reserves"]))
print("base buffer: %s" %(trades.iloc[2]["base_buffer"]))
print("bond buffer: %s" %(trades.iloc[2]["bond_buffer"]))
print("share price: %s" %(trades.iloc[2]["share_price"]))
print("fixed apr: %s" %(trades.iloc[2]["fixed_apr"]))

# print(trades.iloc[2]["base_buffer"])
# print(trades.iloc[1]["agent_1_total_longs"])
# print(trades.iloc[1]["fixed_apr"])
# print(trades.keys())

before values: 

bond reserves: 313282.9751069175
share reserves: 2000000.0
base buffer: 0.0
bond buffer: 0
share price: 1.0
fixed apr: 0.030000000000000325 

after values: 

bond reserves: 312275.9597442398
share reserves: 2001000.0
base buffer: 1007.015362677666
bond buffer: 0
share price: 1.0
fixed apr: 0.029902108970280877 

after close: 

bond reserves: 312275.9597442398
share reserves: 2000045.9266431273
base buffer: 0.0
bond buffer: 0
share price: 1.002715973203273
fixed apr: 0.03014308018138059


### Charts

In [201]:
# share_price = trades.iloc[0]["share_price"]
# spot_price = trades.iloc[0]["spot_price"]
# fixed_apr = trades.iloc[0]["fixed_apr"]

# print(share_price)
# print(spot_price)
# print(fixed_apr)

# fig, axes, _ = output_utils.get_gridspec_subplots(nrows=1, ncols=1)
# ax = reserves.plot.scatter(x="target_apr", y="bond_reserves", ax=axes[0], c="blue")
# # ax = trades.iloc[:-1].plot(x="target_apr", y="bond_reserves", ax=axes[0], c="orange")
# ax.set_xlabel("target fixed apr")
# ax.set_ylabel("bond reserves")
# ax.set_title("Reserves vs. Fixed APR")
# fig.set_size_inches(fig_size)
